### Trabalho 3 - Inversores
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado


O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.

   i. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   
   ii. Cada inversor é regido pelas seguintes transformações
$$\mathbf{invert}\mathtt(in,out)$$
$$x \gets \mathsf{read}(\mathtt{in})$$
$$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$            
$$\mathsf{write}(\mathtt{out},s)$$
   iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
   
   iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
   
   iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.
                    
   1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
   2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

### Implementação

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools

def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

### Criação do SFOTS

In [2]:
import numpy
# valor aleatorio entre 0 e 1 para determinar qual a funcao a usar |¬x ∥ s←s⊕x |
funcao = numpy.random.choice([0,1])

def trans1(curr,prox):

    t0 = And(Equals(curr['pc'],Int(0)), Equals(funcao,Int(0)), Equals(prox['pc'],Int(1)), Equals(prox['Inv_B'],Not(curr['Inv_A'])))
    t1 = And(Equals(curr['pc'],Int(0)), Equals(funcao,Int(1)), Equals(prox['pc'],Int(1)), Equals(prox['Inv_B'],Xor(curr['Inv_A'],curr['Inv_B'])))
    t2 = And(Equals(curr['pc'],Int(1)), Equals(funcao,Int(0)), Equals(prox['pc'],Int(2)), Equals(prox['Inv_C'],Not(curr['Inv_B'])))
    t3 = And(Equals(curr['pc'],Int(1)), Equals(funcao,Int(1)), Equals(prox['pc'],Int(2)), Equals(prox['Inv_C'],Xor(curr['Inv_B'],curr['Inv_C'])))
    t4 = And(Equals(curr['pc'],Int(2)), Equals(funcao,Int(0)), Equals(prox['pc'],Int(3)), Equals(prox['Inv_D'],Not(curr['Inv_C'])))
    t5 = And(Equals(curr['pc'],Int(2)), Equals(funcao,Int(1)), Equals(prox['pc'],Int(3)), Equals(prox['Inv_D'],Xor(curr['Inv_C'],curr['Inv_D'])))
    t6 = And(Equals(curr['pc'],Int(3)), Equals(funcao,Int(0)), Equals(prox['pc'],Int(0)), Equals(prox['Inv_A'],Not(curr['Inv_D'])))
    t7 = And(Equals(curr['pc'],Int(3)), Equals(funcao,Int(1)), Equals(prox['pc'],Int(0)), Equals(prox['Inv_A'],Xor(curr['inv_d'],curr['Inv_A'])))

    return Or(t0,t1,t2,t3,t4,t5,t6,t7)
        
def init1(state):
    return And(Equals(state['pc'],Int(0)), 
          Equals(state['Inv_A'],Int(numpy.random.choice([0,1]))),
           Equals(state['Inv_B'],numpy.random.choice(Int(0),Int(1))),
           Equals(state['Inv_C'],numpy.random.choice(Int(0),Int(1))),
           Equals(state['Inv_D'],numpy.random.choice(Int(0),Int(1))),
           Not(And(Equals(inv_a,Int(0)),Equals(inv_b,Int(0)),Equals(inv_c,Int(0)),Equals(inv_d,Int(0))))
            )

def error1(state):
    return And(Equals(state['Inv_A'],Int(0)),
               Equals(state['Inv_B'],Int(0)),
               Equals(state['Inv_C'],Int(0)),
               Equals(state['Inv_D'],Int(0))
              )


In [3]:
def genTrace(vars,init,trans,error,n):
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
                    
genTrace(['Inv_A','Inv_B','Inv_C','Inv_D','pc'],init1,trans1,error1,10)

PysmtTypeError: Invalid type in constant. The type was:<class 'numpy.int64'>

###  Model checking com interpolantes

In [ ]:
def invert(trans):
    return (lambda c, p: trans(p,c))